In [48]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Internal Imports

In [49]:
# Find ".env" file and add the package to $PATH
import os, sys
import typing as t
from typing import TypeAlias
from dotenv import find_dotenv
sys.path.append(os.path.dirname(find_dotenv()))

# Use local package for modularity
import emotion_analysis as ea
import emotion_analysis.data.dataset as data
import emotion_analysis.data.transform as transform
from emotion_analysis.model.emotion_cause_text import load_text_model
from emotion_analysis.model.emotion_cause_text import EmotionCauseTextModel

External Imports

In [50]:
import jax
import jax.numpy as jnp
import jax.random as jrm
from jax import Array
from jax.typing import ArrayLike, DTypeLike
import flax
import flax.linen as nn
import numpy as np
import optax as opt
import mlflow as mlf
from transformers import RobertaConfig, FlaxRobertaModel, RobertaTokenizerFast
from transformers import PretrainedConfig
from torch.utils.data import Dataset, DataLoader, Subset, random_split

In [51]:
# Load pretrained model along with its tokenizer
model, params, tokenizer = load_text_model()

# Select tokenization method
tokenize = transform.Tokenize(tokenizer)
collate = transform.DataTokenizerCollator(tokenize)

# Load and prepare the data
task = 'task_1'
DataSplit: TypeAlias = t.Literal['train', 'valid', 'test']
ds_test = data.ECACDataset(ea.DATA_DIR, task, split='test')
ds_train = data.ECACDataset(ea.DATA_DIR, task, split='train')
ds_train, ds_valid = random_split(ds_train, [0.75, 0.25])
num_classes: int = ds_test.num_emotions

# Split into train-valid-test
dataloader: t.Dict[DataSplit, DataLoader[t.Dict[str, Array]]] = {
    'train': DataLoader(ds_train, 32, True, collate_fn=collate, drop_last=True),
    'valid': DataLoader(ds_valid, 32, False, collate_fn=collate),
    'test': DataLoader(ds_test, 32, False, collate_fn=collate),
}

In [36]:
# Generate random key
key = jrm.key(ea.SEED)
key, init_key = jrm.split(key, 2)


module = EmotionCauseTextModel(text_encoder=model, num_classes=num_classes)
print(module.tabulate(init_key, input_ids=jnp.zeros((33, 93)), attention_mask=jnp.zeros((33, 93))))


                         EmotionCauseTextModel Summary                          
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ path          ┃ module        ┃ inputs        ┃ outputs      ┃ params        ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│               │ EmotionCause… │ attention_ma… │ float32[33,… │               │
│               │               │ float32[33,9… │              │               │
│               │               │ input_ids:    │              │               │
│               │               │ float32[33,9… │              │               │
├───────────────┼───────────────┼───────────────┼──────────────┼───────────────┤
│ text_encoder  │ FlaxRobertaM… │ -             │ attentions:  │               │
│               │               │ float32[33,9… │ None         │               │
│               │               │ -             │ cross_atten… │               │
│               │          